In [71]:
import pandas as pd
import numpy as np

In [290]:
def indice_journee(indice_deb,dataset,columnDateName): # pour déterminer l'indice du dernier match de la journée
    indice_fin = indice_deb
    fondDataset = False #varaible pour determiner si on est à la fin du dataset
    while((not fondDataset) and (abs((dataset[columnDateName].iloc[indice_fin]-dataset[columnDateName].iloc[indice_fin+1]).days)<2)): #tant que l'on a pas un écart de 2 jours entre deux matchs on est dans la même journée
        indice_fin += 1
        if(indice_fin == len(dataset)-1):#si on arrive à la derniere ligne on sort de la boucle
            fondDataset = True #quand on est à la fin on change la variable
    return indice_fin
    

In [358]:
def create_initial_classement(dataset,init_indice,end_indice): #creer le classement initial. Il faut que la premiere journée contienne toutes les équipes pour que sa fonctionne
    classement = {}
    for i in range(init_indice,end_indice+1):
        classementValue = np.arange(20) #liste de classement
        AwayTeam = dataset.loc[i,"AwayTeam"]
        HomeTeam = dataset.loc[i,"HomeTeam"]
        classement[AwayTeam] = [0,classementValue[2*i],dataset.loc[i,"ATG"]]
        classement[HomeTeam] = [0,classementValue[2*i+1],dataset.loc[i,"HTG"]]# 0 car on a 0 points au début de la saison et 2*n pour classement value. poser le sur un papier pour comprendre
        
    return classement

In [359]:
def set_classement(classement,dataset,deb_indice,fin_indice): #met à jour le classement
    for row in range(deb_indice,fin_indice+1): #dans un premier temps on calcul les points et les buts pour toutes les équipes
        HomeTeam = dataset.loc[row,"HomeTeam"] 
        AwayTeam = dataset.loc[row,"AwayTeam"]
        classement[HomeTeam][0] = 3*dataset.loc[row,"HTW"] + dataset.loc[row,"HTD"]
        classement[AwayTeam][0] = 3*dataset.loc[row,"ATW"] + dataset.loc[row,"ATD"]
        classement[HomeTeam][2] = dataset.loc[row,"HTG"]
        classement[AwayTeam][2] = dataset.loc[row,"ATG"]
        
    for key1 in classement : #on remet le classement dans l'ordre
        tableTeam = classement[key1]
        rank = 0
        for key2 in classement:
            if(key2 != key1):
                tableOtherTeam = classement[key2]
                if (tableOtherTeam[0]>=tableTeam[0]):
                    if(tableOtherTeam[2]>tableTeam[2]):
                        rank += 1
        classement[key1][1] = rank
    return classement
            

In [353]:
def fillUpDataset(dataset,classement,deb_indice,fin_indice): #remplit le dataset journée par journée
    rankValue = [] #on met tous les classement dans une liste pour calculer les quartiles
    for key in classement : 
        rankValue.append(classement[key][1])
    quartile1 = np.percentile(rankValue,35) # on a elargit la largueur des quartiles 
    quartile3 = np.percentile(rankValue,65)
    for i in range(deb_indice,fin_indice+1): #on remplit les lignes une à une
        HomeTeamName = dataset.loc[i,"HomeTeam"]
        AwayTeamName = dataset.loc[i,"AwayTeam"]
        if(classement[HomeTeamName][1] < quartile1):
            dataset.loc[i,"HRP"] = "HautDeTableau"
        if(classement[AwayTeamName][1] < quartile1):
            dataset.loc[i,"ARP"] = "HautDeTableau"
            
        if(classement[HomeTeamName][1] > quartile3):
            dataset.loc[i,"HRP"] = "BasDeTableau"
        if(classement[AwayTeamName][1] > quartile3):
            dataset.loc[i,"ARP"] = "BasDeTableau"
            
        if ((quartile3>=classement[HomeTeamName][1]>=quartile1)) : 
            dataset.loc[i,"HRP"] = "MilieuDeTableau"
        if ((quartile3>=classement[AwayTeamName][1]>=quartile1)) : 
            dataset.loc[i,"ARP"] = "MilieuDeTableau"

In [327]:
#on créer les colonnes ranking pour hometeam et awawyteam
def create_ranking(dataset):
    dataset["HRP"]=[0]*len(dataset) #on intialise les colonnes
    dataset['ARP']=[0]*len(dataset)
    deb_indice = 0
    fin_indice = indice_journee(deb_indice,dataset,"DateTime")
    classement= create_initial_classement(dataset,deb_indice,fin_indice)
    fillUpDataset(dataset,classement,deb_indice,fin_indice)
    while(fin_indice != len(dataset)-1): #tant que l'on est pas à la fin du dataset on remplit le tableau
        deb_indice = fin_indice + 1
        fin_indice = indice_journee(deb_indice,dataset,"DateTime")
        classement= set_classement(classement,dataset,deb_indice,fin_indice)
        fillUpDataset(dataset,classement,deb_indice,fin_indice)

In [492]:
df_2015_2016 = pd.read_csv("../cleanedDataset/2014_2015.csv")
#df_2015_2016.drop(len(df_2015_2016)-1,0, inplace=True)  #à rajouter pour les saison 2013/2014 et 2015/2016 car il y a une ligne en trop
df_2015_2016['DateTime'] = pd.to_datetime(df_2015_2016["Date"], format = "%d/%m/%y")
df_2015_2016.drop(columns=["Date","Unnamed: 0"])
create_ranking(df_2015_2016)
df_2015_2016.head(50)
final_df = df_2015_2016.drop(columns=["Unnamed: 0"])
final_df.head()

#LE code ne marche pas pour la saison 20202021 car la premiere journée ne contient pas toutes équipes

,Div,Date,HomeTeam,AwayTeam,FTR,HTS,HTST,HTW,HTD,HTL,...,ATW,ATD,ATL,ATF,ATY,ATR,ATG,DateTime,HRP,ARP
0,F1,08/08/14,Reims,Paris SG,1.0,0.0,0.0,0,0,0,...,0,0,0,0.0,0.0,0.0,0.0,2014-08-08,HautDeTableau,HautDeTableau
1,F1,09/08/14,Bastia,Marseille,1.0,0.0,0.0,0,0,0,...,0,0,0,0.0,0.0,0.0,0.0,2014-08-09,HautDeTableau,HautDeTableau
2,F1,09/08/14,Evian Thonon Gaillard,Caen,0.0,0.0,0.0,0,0,0,...,0,0,0,0.0,0.0,0.0,0.0,2014-08-09,HautDeTableau,HautDeTableau
3,F1,09/08/14,Guingamp,St Etienne,0.0,0.0,0.0,0,0,0,...,0,0,0,0.0,0.0,0.0,0.0,2014-08-09,MilieuDeTableau,HautDeTableau
4,F1,09/08/14,Lille,Metz,1.0,0.0,0.0,0,0,0,...,0,0,0,0.0,0.0,0.0,0.0,2014-08-09,MilieuDeTableau,MilieuDeTableau


In [487]:
final_df.to_csv("datasetWithRanking/2019_2020_Rank.csv")